# 时间处理基础



Pandas提供了四种类型的生成日期时间的对象：日期时间、时间增量、时间跨度、日期偏移量

（1） 日期时间(Date Times)：具有时区支持的特定日期和时间。与Python标准库中的datetime.datetime类似。如2020年12月6日13点37分50秒；

（2） 时间增量(Time Deltas)：绝对持续时间，用于在指定时间点基础上增加指定的增量，如在某年月日的基础上增加2天、增加2个月、减少4小时等，最后产生一个新的时间点；

（3） 时间跨度(Time Span)：由时间点及其相关周期定义的时间跨度，如连续产生一年四个季度的时间序列；

（4） 日期偏移(Date Offsets)：以日历计算的相对持续时间，表示时间间隔，两个时间点之间的长度，如日、周、月、季度、年。

<img src="images/20220415102506.png">

一般情况下，时间序列主要是 `Series`  或 `DataFrame` 的时间型索引，可以用时间元素进行操控。

In [ ]:
import pandas as pd

import numpy as np

import time

import datetime

## 一、时间序列类型

### 1.时间戳

python datetime的替代品

时间戳相当于python的Datetime，在大多数情况下可以与之互换。

该类型用于组成DatetimeIndex的条目，以及pandas中其他面向时间序列的数据结构。

#### 例子:
- 转换类似日期时间的字符串

In [ ]:
pd.Timestamp('2022-01-01 12')

In [ ]:
pd.Timestamp('2022-01')

- unit参数为s:
       这将转换以秒为单位表示Unix历元的浮点值 
       1970年1月1日这个时间正是Unix系统的起始时间

In [ ]:
pd.Timestamp(time.time(), unit="s")

- year, month, day,hour, minute, second, microsecond单独设置时间

In [ ]:
pd.Timestamp(2022, 1, 1, 12)

In [ ]:
pd.Timestamp(year=2022, month=1, day=1, hour=12)

In [ ]:
总结:
    
Timestamp

- 将字符串或者unix时间转化为 pandas对应的时间戳

### 2.时间间隔--实现datetime加减

表示持续时间，即两个日期或时间之间的差异。

相当于python的datetime.timedelta，在大多数情况下可以与之互换

In [ ]:
ts = pd.Timestamp('2022-01-01 12')
ts + pd.Timedelta(-1, "d")

In [ ]:
#时间间隔
td = pd.Timedelta(days=5, minutes=50, seconds=20)
td

In [ ]:
ts - td

In [ ]:
td.total_seconds()

In [ ]:
from datetime import timedelta

now = datetime.datetime.now()
print(now)
#计算当前时间往后100天的日期
#dt = now + pd.Timedelta(days=100)
dt = now + timedelta(days=100)
print(dt,type(dt))
#只显示年月日
dt.strftime('%Y-%m-%d')

### 3.时间转化

`to_datetime` 转换时间戳

你可能会想到，我们经常要和文本数据（字符串）打交道，能否快速将文本数据转为时间戳呢？

      答案是可以的，通过 to_datetime 能快速将字符串转换为时间戳。当传递一个Series时，它会返回一个Series（具有相同的索引），而类似列表的则转换为DatetimeIndex。
      
`to_datetime(arg, errors='raise', dayfirst=False, yearfirst=False, utc=None, format=None, unit=None, infer_datetime_format=False, origin='unix')`

函数用户将数组、序列或dict的对象转换为datetime对象

- `arg` 要转换为日期时间的对象
- `errors`:错误处理
        If 'raise',将引发异常.
        If 'coerce', 无效的转换,使用NaT.
        If 'ignore', 无效的转换,将使用输入的数据.
- `dayfirst` :转换时指定日期分析顺序
- `utc` :控制与时区相关的解析、本地化和转换(忽略)
- `format`: 用于分析时间的strftime，例如“%d/%m/%Y”,自定义格式
- `unit` : D,s,ms 将时间戳转datetime
- `origin`: 定义参考日期。数值将被解析为自该参考日期起的单位数

1). 处理各种输入格式

    从一个数据帧的多个列中组装日期时间。

    这些键可以是常见的缩写，

    如['year'、'month'、'day'、'minute'、'second'、'ms'、'us'、'ns']），

    也可以是相同的复数形式

In [ ]:
df = pd.DataFrame({'year': [2015, 2016],'month': [2, 3],'day': [4, 5]})
df

In [ ]:
pd.to_datetime(df)

2). 将字符串转datetime

In [ ]:
pd.to_datetime(['11-12-2021'])

In [ ]:
pd.to_datetime(["2005/11/23", "2010.12.31"])

3). 除了可以将文本数据转为时间戳外，还可以将 unix 时间转为时间戳。

In [ ]:
pd.to_datetime([1349720105, 1349806505, 1349892905], unit="s")

In [ ]:
pd.to_datetime([1349720105100, 1349720105200, 1349720105300], unit="ms")

4). 自动识别异常
    - 210605

In [ ]:
pd.to_datetime('210605')

In [ ]:
pd.to_datetime('210605',yearfirst=True)

5). 配合unit参数,使用非unix时间

In [ ]:
# origin参考起始时间
pd.to_datetime([1, 2, 3], unit='D', origin=pd.Timestamp('2020-01-11'))

6).  不可转换日期/时间 

        如果日期不符合时间戳限制，则传递errors='ignore'将返回原始输入，而不是引发任何异常。

        除了将非日期（或不可解析的日期）强制传递给NaT之外，传递errors='coerce'还会将越界日期强制传递给NaT

- errors`:错误处理
        If 'raise',将引发异常.
        If 'coerce', 无效的转换,使用NaT.
        If 'ignore', 无效的转换,将使用输入的数据.

In [ ]:
pd.to_datetime('120211204')

In [ ]:
pd.to_datetime('120211204',errors="ignore")

In [ ]:
pd.to_datetime('120211204',errors="coerce")

In [ ]:
pd.to_datetime(pd.Series(["Jul 31, 2018", "2018-05-10", None]))

> 对于float arg，可能会发生精确舍入。要防止意外行为，请使用固定宽度的精确类型。

### 4.生成时间戳范围


有时候，我们可能想要生成某个范围内的时间戳。例如，我想要生成 "2018-6-26" 这一天之后的8天时间戳

我们可以使用 `date_range` 和 `bdate_range` 来完成时间戳范围的生成。

#### date_range() 返回固定频率的DatetimeIndex
`date_range(start=None, end=None, periods=None, freq=None, tz=None, normalize=False, name=None, closed=None, **kwargs)`

 返回等距时间点的范围（其中任意两个相邻点之间的差值由给定频率指定），以便它们都满足开始<[=]x<[=]end，其中第一个和最后一个分别为。，该范围内的第一个和最后一个时间点位于freq的边界（如果以频率字符串的形式给出）或对freq有效
- start :生成日期的左边界
- end : 生成日期的左边界
- periods: 要生成的周期数 

- freq : 频率, default 'D' ,频率字符串可以有倍数, '5H'
- tz: 时区用于返回本地化日期时间索引的时区名称，例如“Asia/Hong_Kong”。默认情况下，生成的DatetimeIndex是时区初始索引。
- normalize: 默认 False, 在生成日期范围之前，将开始/结束日期标准化
- name: 默认 None 设置返回DatetimeIndex name   
- closed: 控制是否包括屏幕上的“开始”和“结束”边界默认设置包括两端的边界点{None, ‘left’, ‘right’}

    (closed 1.4.0被启用) inclusive:包括边界{"both", "neither", "left", "right"}, default "both"
 

1). 指定值
- 默认是包含开始和结束时间, 默认频率使用的D(天)

In [ ]:
pd.date_range(start='1/1/2021', end='1/08/2021')

In [ ]:
pd.date_range(start='2010', end='2011')

2). 指定开始日期,设置期间数

In [ ]:
pd.date_range(start='1/1/2018', periods=8)

- 指定开始、结束和期间；频率自动生成（线性间隔）

In [ ]:
pd.date_range(start='2018-04-24', end='2018-04-27', periods=3)

In [ ]:
pd.date_range(start='2018-04-24', end='2018-04-27', periods=4)

3). 频率 freq
<img src="images/20220417232702.png" style="width:80%">

- 月缩写：JAN/FEB/MAR/APR/MAY/JUN/JUL/AUG/SEP/OCT/NOV/DEC
- 星期几缩写：MON/TUE/WED/THU/FRI/SAT/SUN
- 所以Q-月只有三种情况：1-4-7-10,2-5-8-11,3-6-9-12

In [ ]:
pd.date_range('2022/1/1','2022/2/1', freq = 'W-MON')
# W-MON：从指定星期几开始算起，每周
# 星期几缩写：MON/TUE/WED/THU/FRI/SAT/SUN

In [ ]:
pd.date_range('2021/1/1','2021/5/1', freq = 'WOM-2MON')
# WOM-2MON：每月的第几个星期几开始算，这里是每月第二个星期一

In [ ]:
# 默认freq = 'D'：每日历日
pd.date_range('2022/1/1','2022/1/4')

In [ ]:
pd.date_range('2022/1/1','2022/1/5', freq = 'B')  # B：每工作日

In [ ]:
pd.date_range('2022/1/1','2022/1/2', freq = 'H')  # H：每小时

In [ ]:
pd.date_range('2022/1/1 12:00','2022/1/1 12:10', freq = 'T')  # T/MIN：每分

In [ ]:
print(pd.date_range('2022/1/1 12:00:00','2022/1/1 12:00:10', freq = 'S'))  # S：每秒

In [ ]:
print(pd.date_range('2022/1/1 12:00:00','2022/1/1 12:00:10', freq = 'L'))  # L：每毫秒（千分之一秒）

In [ ]:
pd.date_range('2022/1/1 12:00:00','2022/1/1 12:00:10', freq = 'U')  # U：每微秒（百万分之一秒）

In [ ]:
# M：每月最后一个日历日
pd.date_range('2017','2018', freq = 'M')

In [ ]:
# Q-月：指定月为季度末，每个季度末最后一月的最后一个日历日
print(pd.date_range('2017','2020', freq = 'Q-DEC')) 

In [ ]:
# A-月：每年指定月份的最后一个日历日
print(pd.date_range('2017','2020', freq = 'A-DEC')) 

In [ ]:
# B,(M,Q,A),S 	分别代表了工作日，（以月为频率，以季度为频率，以年为频率），最接近月初的那一天
print(pd.date_range('2017','2018', freq = 'BM'))  
print(pd.date_range('2017','2020', freq = 'BQ-DEC'))  
print(pd.date_range('2017','2020', freq = 'BA-DEC')) 
print('------')
# BM：每月最后一个工作日
# BQ-月：指定月为季度末，每个季度末最后一月的最后一个工作日
# BA-月：每年指定月份的最后一个工作日

In [ ]:
print(pd.date_range('2017','2018', freq = 'MS'))  
print(pd.date_range('2017','2020', freq = 'QS-DEC'))  
print(pd.date_range('2017','2020', freq = 'AS-DEC')) 
print('------')
# M：每月第一个日历日
# Q-月：指定月为季度末，每个季度末最后一月的第一个日历日
# A-月：每年指定月份的第一个日历日

In [ ]:
print(pd.date_range('2017','2018', freq = 'BMS'))  
print(pd.date_range('2017','2020', freq = 'BQS-DEC'))  
print(pd.date_range('2017','2020', freq = 'BAS-DEC')) 
print('------')
# BM：每月第一个工作日
# BQ-月：指定月为季度末，每个季度末最后一月的第一个工作日
# BA-月：每年指定月份的第一个工作日

In [ ]:
# pd.date_range()-日期范围：复合频率

print(pd.date_range('2017/1/1','2017/2/1', freq = '7D'))  # 7天
print(pd.date_range('2017/1/1','2017/1/2', freq = '2h30min'))  # 2小时30分钟
print(pd.date_range('2017','2018', freq = '2M'))  # 2月，每月最后一个日历日

In [ ]:
pd.date_range(start='1/1/2018', periods=5, freq='3M')

4).  `closed`: 边界设置

        控制是否包括屏幕上的“开始”和“结束”边界默认设置包括两端的边界点{None, ‘left’, ‘right’}

In [ ]:
pd.date_range(start='2022-01-01', end='2022-01-04', closed=None)

In [ ]:
pd.date_range(start='2022-01-01', end='2022-01-04')

如果需要排除"end",则使用“closed='left'”

In [ ]:
pd.date_range(start='2022-01-01', end='2022-01-04',closed="left")

如果需要排除"start",则使用“closed='right'”

In [ ]:
pd.date_range(start='2022-01-01', end='2022-01-04',closed="right")

5)  `normalize`: 在生成日期范围之前，将开始/结束日期标准化

In [ ]:
pd.date_range(start = '1/1/2021 15:30', periods =10, name = 'mypd')

In [ ]:
pd.date_range(start = '1/1/2021 15:30', periods =10, name = 'mypd',normalize=True)

In [ ]:
pd.date_range(start = '1/1/2021 15:30', periods=10, name='mypd', freq="H")

In [ ]:
pd.date_range(start = '1/1/2021 15:30', periods=10, name='mypd',normalize=True, freq="H")

## 5.时期频率转换

In [ ]:
# asfreq：时期频率转换

ts = pd.Series(np.random.rand(4),
              index = pd.date_range('20170101','20170104'))
print(ts)
print(ts.asfreq('4H',method = 'ffill'))
# 改变频率，这里是D改为4H
# method：插值模式，None不插值，ffill用之前值填充，bfill用之后值填充

## 6. shift()-时间频率进行移位

`shift(periods=1, freq=None, axis=0, fill_value=None)`
按所需的时段数和可选的时间频率进行移位索引。

如果未传递freq，则在不重新调整数据的情况下移动索引。如果传递了freq（在这种情况下，索引必须是date或datetime，否则将引发NotImplementedError），只要在索引中设置了freq或推断的_freq属性，就可以推断freq

- periods: 要转换的时段数。可以是正面的，也可以是负面的
- freq:如果指定了freq，则索引值会移位，但数据不会重新对齐。也就是说，如果要在移动时扩展索引并保留原始数据
- axis:{0 or 'index', 1 or 'columns', None} 转换方向.
- fill_value: 填充值

In [ ]:
df = pd.DataFrame(np.random.rand(16).reshape((4,4)), 
                  index = pd.date_range('20210101','20210104'),
                  columns=list('ABCD'))
df

In [ ]:
# 正数：数值后移（滞后）,模式为行
df.shift(periods=2)

In [ ]:
# 正数：数值后移（滞后） 设置为列
df.shift(periods=1, axis="columns")

In [ ]:
# 正数: 数值后移, NaN填充为0
df.shift(periods=3, fill_value=0)

In [ ]:
# 当设置freq时 ,对时间索引移动
df.shift(periods=3, freq="D")

In [ ]:
# 计算变化百分比，这里计算：该时间戳与上一个时间戳相比，变化百分比
per = ts/ts.shift(1) - 1
print(per)

# 作业

作业1：请输出以下时间序列
<img src="images/20220417232940.png" style="width:60%"/>
